In [1]:
# https://k3no.medium.com/how-to-query-in-graphql-6ebb3f7085dc
# https://github.com/sushiswap/sushiswap-subgraph

In [2]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd

mainnet = 'https://api.thegraph.com/subgraphs/name/jiro-ono/sushiswap-exchange'
fantom = 'https://api.thegraph.com/subgraphs/name/sushiswap/fantom-exchange'
matic = 'https://api.thegraph.com/subgraphs/name/sushiswap/matic-exchange'

## Matic

In [4]:
sample_transport=RequestsHTTPTransport(
    url=matic,
    verify=True,
    retries=3,
)

client = Client(transport=sample_transport)

In [5]:
query = gql('''
query {
  factories(where: {liquidityUSD_gt:0}) {
    pairCount
  }
}
''')

response = client.execute(query)
number_of_pairs = int(response['factories'][0]['pairCount'])
print(number_of_pairs)

851


In [10]:
df = pd.DataFrame([])
response_dict = {}
response_list = []
id_list = []


for i in range(number_of_pairs):
    if i % 100 == 0:
        querystring = '''
              query {
              pairs(first: '''+str(100)+' skip: '+str(i)+''' where: {volumeUSD_gt:0}){
                      name
                      id
                  }
                }
                '''
        response = client.execute(gql(querystring))
        for row in response['pairs']:
            response_list.append(row['name'])
            id_list.append(row['id'])    

df = pd.DataFrame.from_dict(response_list)
df[['Token 0', 'Token 1']] = df[0].str.split('-', expand = True, n = 1)
df.value_counts()
df

,0,Token 0,Token 1
0,YELD-WMATIC,YELD,WMATIC
1,WMATIC-YELD,WMATIC,YELD
2,WMATIC-MOON,WMATIC,MOON
3,WETH-IRON,WETH,IRON
4,WMATIC-DRC,WMATIC,DRC
...,...,...,...
435,USDC-JPYC,USDC,JPYC
436,ET-WETH,ET,WETH
437,pDAI-USDT,pDAI,USDT
438,USDC-AMIS,USDC,AMIS


In [11]:
# interesting_tokens = ['YFI', 'WOOFY', 'CRV', 'ILV', 'WBTC', 'WMATIC', 'MATIC', 'AAVE', 'WETH', 'ETH', 'SUSHI', 'UNI', 'SNX', 'LINK', 'DAI', 'USDC', 'USDT', 'FRAX']
# interesting_tokens = ['USDC', 'USDT', 'DAI', 'FRAX', 'IRON']
interesting_tokens = ['MATIC', 'DAI', 'USDC', 'USDT', 'WBTC', 'WETH', 'WMATIC']

criteria = df['Token 0'].isin(interesting_tokens) & \
           df['Token 1'].isin(interesting_tokens)

df_filtered = df.loc[criteria,:]
df_filtered.columns = ['Pair', 'Token 0', 'Token 1']
df_filtered.head()

,Pair,Token 0,Token 1
87,USDC-WETH,USDC,WETH
97,DAI-USDT,DAI,USDT
125,USDC-USDT,USDC,USDT
142,WMATIC-USDT,WMATIC,USDT
177,WETH-DAI,WETH,DAI


In [12]:
# make a list of strings with double quotes

stringtest = "["

for i in df_filtered['Pair'].values.tolist():
    stringtest += '"'+ i + '",'
    
stringtest[:-1]+"]"

'["USDC-WETH","DAI-USDT","USDC-USDT","WMATIC-USDT","WETH-DAI","WMATIC-WBTC","WMATIC-DAI","WETH-USDT","WMATIC-WETH","WMATIC-USDC","USDC-DAI","WBTC-USDC","WBTC-WETH"]'

In [8]:
querystring = '''
              query {
  pairs(where: {name_in: ''' + stringtest[:-1]+"]" + '''}){
    name
    dayData{
      date
      reserveUSD
      volumeUSD
      txCount
    }
  }
}
                '''
response = client.execute(gql(querystring))
response

{'pairs': [{'dayData': [{'date': 1621296000,
     'reserveUSD': '2134.040263710702641917271382309285',
     'txCount': '11',
     'volumeUSD': '612.6849219683397147376470863620611'},
    {'date': 1621382400,
     'reserveUSD': '5095343.794480396319059345248991458',
     'txCount': '1046',
     'volumeUSD': '2353509.179360423566738388574263848'},
    {'date': 1621468800,
     'reserveUSD': '8463147.210895523987001164897695253',
     'txCount': '255',
     'volumeUSD': '613719.6151607147548550266710408844'},
    {'date': 1621555200,
     'reserveUSD': '9128434.710236544022353138801699636',
     'txCount': '218',
     'volumeUSD': '968980.6519168736313512314844739305'},
    {'date': 1621641600,
     'reserveUSD': '8811008.909905263086234836307299904',
     'txCount': '179',
     'volumeUSD': '796978.7937894434241363937829997433'},
    {'date': 1621728000,
     'reserveUSD': '7042581.616448778504439094289549729',
     'txCount': '175',
     'volumeUSD': '435016.4777905333476112728197813453

In [9]:
pd.to_datetime(1615226400, unit='s')

Timestamp('2021-03-08 18:00:00')

In [10]:
df4 = pd.DataFrame([])

for pair_no in range(len(response['pairs'])):
    df3 = pd.DataFrame.from_dict(response['pairs'][pair_no])
    df3[['date', 'reserveUSD', 'txCount', 'volumeUSD']] = df3['dayData'].apply(pd.Series)
    df3 = df3.drop(columns = ['dayData'])
    df4 = df4.append(df3)
    
df4[['date','reserveUSD','volumeUSD']] = df4[['date','reserveUSD','volumeUSD']].astype(float)
df4['txCount'] = df4['txCount'].astype(int)
df4['date'] = pd.to_datetime(df4['date'], unit='s')
df4['fee'] = round(df4['volumeUSD'] * 0.003,6)
df4['1y APR for 100 invested'] = round((100/df4['reserveUSD']) * df4['fee'] * 365,3)
df4['name'] = '[Matic] ' + df4['name']
df4.head(1)

,name,date,reserveUSD,txCount,volumeUSD,fee,1y APR for 100 invested
0,[Matic] SNX-WETH,2021-05-18,2134.040264,11,612.684922,1.838055,31.438


In [11]:
latest_results = df4.drop_duplicates(subset = ['name'], keep = 'last')\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)
latest_results.style.format({'reserveUSD': "{:0<4,.2f}"})

,name,date,reserveUSD,txCount,volumeUSD,fee,1y APR for 100 invested
0,[Matic] WBTC-AAVE,2021-06-07 00:00:00,0.00,4,1.416281,0.004249,226893.087000
1,[Matic] WETH-WOOFY,2021-06-03 00:00:00,0.01,3,28.149906,0.084450,218792.699000
2,[Matic] FRAX-USDT,2021-06-09 00:00:00,0.03,11,10.816569,0.032450,41700.428000
3,[Matic] WMATIC-USDC,2021-06-11 00:00:00,"19,897,203.69",29164,59686302.314898,179058.906945,328.471000
4,[Matic] WMATIC-USDT,2021-06-14 00:00:00,"1,580.79",1473,3041.217489,9.123652,210.662000
5,[Matic] WMATIC-WETH,2021-06-09 00:00:00,"107,111,717.56",15652,49749848.734405,149249.546203,50.859000
6,[Matic] USDC-WETH,2021-06-11 00:00:00,"135,360,616.42",9637,30860221.998012,92580.665994,24.964000
7,[Matic] SUSHI-WMATIC,2021-06-16 00:00:00,"282,892.30",517,21652.492225,64.957477,8.381000
8,[Matic] WMATIC-YFI,2021-06-15 00:00:00,1.65,3,0.124082,0.000372,8.239000
9,[Matic] WMATIC-WBTC,2021-06-16 00:00:00,"19,052.07",90,1104.647739,3.313943,6.349000


In [17]:
df_horizontal = df4.pivot(index = 'name',
                          columns = 'date',
                          values = '1y APR for 100 invested')
# .style.format("{:0<4,.2f}")

# drop last day as it is incomplete
df_horizontal = df_horizontal.drop(columns = df_horizontal.columns[-1])
df_horizontal

df_horizontal['Median APR last day'] = df_horizontal[df_horizontal.columns[-1:]].median(axis = 1)
df_horizontal['Median APR last 3 days'] = df_horizontal[df_horizontal.columns[-3:]].median(axis = 1)
df_horizontal['Median APR last 7 days'] = df_horizontal[df_horizontal.columns[-7:]].median(axis = 1)
df_horizontal['Median APR last 14 days'] = df_horizontal[df_horizontal.columns[-14:]].median(axis = 1)
df_horizontal[['Median APR last day','Median APR last 3 days', 'Median APR last 7 days', 'Median APR last 14 days']]\
.sort_values(by = 'Median APR last day', ascending = False)

date,Median APR last day,Median APR last 3 days,Median APR last 7 days,Median APR last 14 days
name,,,,
[Matic] WMATIC-WBTC,59.198,59.198,57.329,47.3945
[Matic] SUSHI-WMATIC,42.842,42.842,43.329,43.3290
[Matic] WMATIC-SNX,33.882,33.882,33.882,33.8820
[Matic] SUSHI-WETH,31.492,31.492,31.492,31.4920
[Matic] WMATIC-AAVE,29.537,29.537,29.537,26.5895
[Matic] WMATIC-DAI,29.146,29.146,38.050,38.7755
[Matic] SUSHI-USDC,28.493,28.493,34.483,34.4830
[Matic] WMATIC-LINK,28.403,28.403,28.403,125.0535
[Matic] WMATIC-WOOFY,25.084,25.084,25.084,8.1555


In [ ]:




# make a list of strings with double quotes
stringtest = "["

for i in df_filtered['Pair'].values.tolist():
    stringtest += '"'+ i + '",'

# find the daily returns
querystring = '''
              query {
  pairs(where: {name_in: ''' + stringtest[:-1]+"]" + '''}){
    name
    dayData{
      date
      reserveUSD
      volumeUSD
      txCount
    }
  }
}
                '''
response = client.execute(gql(querystring))


# turn the daily returns into a dataframe
df4_fantom = pd.DataFrame([])

for pair_no in range(len(response['pairs'])):
    df3_fantom = pd.DataFrame.from_dict(response['pairs'][pair_no])
    df3_fantom[['date', 'reserveUSD', 'txCount', 'volumeUSD']] = df3_fantom['dayData'].apply(pd.Series)
    df3_fantom = df3_fantom.drop(columns = ['dayData'])
    df4_fantom = df4_fantom.append(df3_fantom)
    
df4_fantom[['date','reserveUSD','volumeUSD']] = df4_fantom[['date','reserveUSD','volumeUSD']].astype(float)
df4_fantom['txCount'] = df4_fantom['txCount'].astype(int)
df4_fantom['date'] = pd.to_datetime(df4_fantom['date'], unit='s')
df4_fantom['fee'] = round(df4_fantom['volumeUSD'] * 0.003,6)
df4_fantom['1y APR for 100 invested'] = round((100/df4_fantom['reserveUSD']) * df4_fantom['fee'] * 365,3)
df4_fantom['name'] = '[Fantom] ' + df4_fantom['name']

In [44]:
df4_fantom

,name,date,reserveUSD,txCount,volumeUSD,fee,1y APR for 100 invested
0,[Fantom] USDC-DAI,2021-03-03,3721.934410,8,42.983213,0.128950,1.265
1,[Fantom] USDC-DAI,2021-03-04,3696.016205,43,865.346072,2.596038,25.637
2,[Fantom] USDC-DAI,2021-03-05,3772.514951,48,2172.597593,6.517793,63.061
3,[Fantom] USDC-DAI,2021-03-06,3869.306647,14,381.030728,1.143092,10.783
4,[Fantom] USDC-DAI,2021-03-07,3710.734970,20,340.005498,1.020016,10.033
...,...,...,...,...,...,...,...
74,[Fantom] USDC-DAI,2021-06-12,439.759993,5,10.531554,0.031595,2.622
75,[Fantom] USDC-DAI,2021-06-13,455.694886,1,0.180924,0.000543,0.043
76,[Fantom] USDC-DAI,2021-06-15,358.924500,2,0.019737,0.000059,0.006
77,[Fantom] USDC-DAI,2021-06-17,286.423006,4,5.994819,0.017984,2.292


In [40]:
print(df_fantom['Token 1'].value_counts().to_string())

WFTM                                                                                             57
FUSD                                                                                             17
DAI                                                                                              13
BITB                                                                                              4
ICE                                                                                               4
ETH                                                                                               4
SUSHI                                                                                             3
HDogoo                                                                                            3
oho                                                                                               2
UBI                                                                                               2


In [15]:
df4_fantom['date'].max()

Timestamp('2021-06-18 00:00:00')

In [ ]:
df_horizontal_fantom = df4_fantom.pivot(index = 'name',
                          columns = 'date',
                          values = '1y APR for 100 invested')
# .style.format("{:0<4,.2f}")

# drop last day as it is incomplete
df_horizontal_fantom = df_horizontal_fantom.drop(columns = df_horizontal_fantom.columns[-1])
df_horizontal_fantom


df_horizontal_fantom['Median APR last 3 days'] = df_horizontal_fantom[df_horizontal_fantom.columns[-3:]].median(axis = 1)
df_horizontal_fantom['Median APR last 7 days'] = df_horizontal_fantom[df_horizontal_fantom.columns[-7:]].median(axis = 1)
df_horizontal_fantom['Median APR last 14 days'] = df_horizontal_fantom[df_horizontal_fantom.columns[-14:]].median(axis = 1)
df_horizontal_fantom[['Median APR last 3 days', 'Median APR last 7 days', 'Median APR last 14 days']]\
.sort_values(by = 'Median APR last 3 days', ascending = False)

In [ ]:
df_horizontal.append(df_horizontal_fantom)[['Median APR last 3 days', 'Median APR last 7 days', 'Median APR last 14 days']]\
.sort_values(by = 'Median APR last 3 days', ascending = False)

In [ ]:
print(df_horizontal.append(df_horizontal_fantom)[['Median APR last 3 days', 'Median APR last 7 days', 'Median APR last 14 days']]\
.sort_values(by = 'Median APR last 3 days', ascending = False).to_string())

In [ ]:
for pair in latest_results['name'].unique():
    criteria = df4.append(df4_fantom)['name'] == pair
    df4.append(df4_fantom).loc[criteria, ['date', '1y APR for 100 invested']].plot(x ='date',
                                                                y='1y APR for 100 invested',
                                                                kind = 'line',
                                                                title = pair,
                                                                figsize = (17,8),
                                                                ylim = (0,100),
                                                                grid = True)

In [ ]:
https://thegraph.com/explorer/subgraph/sushiswap/matic-bentobox

In [17]:
df4_fantom

,name,date,reserveUSD,txCount,volumeUSD,fee,1y APR for 100 invested
0,[Fantom] USDC-DAI,2021-03-03,3721.934410,8,42.983213,0.128950,1.265
1,[Fantom] USDC-DAI,2021-03-04,3696.016205,43,865.346072,2.596038,25.637
2,[Fantom] USDC-DAI,2021-03-05,3772.514951,48,2172.597593,6.517793,63.061
3,[Fantom] USDC-DAI,2021-03-06,3869.306647,14,381.030728,1.143092,10.783
4,[Fantom] USDC-DAI,2021-03-07,3710.734970,20,340.005498,1.020016,10.033
...,...,...,...,...,...,...,...
74,[Fantom] USDC-DAI,2021-06-12,439.759993,5,10.531554,0.031595,2.622
75,[Fantom] USDC-DAI,2021-06-13,455.694886,1,0.180924,0.000543,0.043
76,[Fantom] USDC-DAI,2021-06-15,358.924500,2,0.019737,0.000059,0.006
77,[Fantom] USDC-DAI,2021-06-17,286.423006,4,5.994819,0.017984,2.292


In [33]:
df4_fantom['name']

0     [Fantom] USDC-DAI
1     [Fantom] USDC-DAI
2     [Fantom] USDC-DAI
3     [Fantom] USDC-DAI
4     [Fantom] USDC-DAI
            ...        
74    [Fantom] USDC-DAI
75    [Fantom] USDC-DAI
76    [Fantom] USDC-DAI
77    [Fantom] USDC-DAI
78    [Fantom] USDC-DAI
Name: name, Length: 79, dtype: object

In [32]:
df4_fantom.sort_values(by = 'date',
                                 ascending = False)

,name,date,reserveUSD,txCount,volumeUSD,fee,1y APR for 100 invested
78,[Fantom] USDC-DAI,2021-06-18,274.620158,2,4.991074,0.014973,1.990
77,[Fantom] USDC-DAI,2021-06-17,286.423006,4,5.994819,0.017984,2.292
76,[Fantom] USDC-DAI,2021-06-15,358.924500,2,0.019737,0.000059,0.006
75,[Fantom] USDC-DAI,2021-06-13,455.694886,1,0.180924,0.000543,0.043
74,[Fantom] USDC-DAI,2021-06-12,439.759993,5,10.531554,0.031595,2.622
...,...,...,...,...,...,...,...
4,[Fantom] USDC-DAI,2021-03-07,3710.734970,20,340.005498,1.020016,10.033
3,[Fantom] USDC-DAI,2021-03-06,3869.306647,14,381.030728,1.143092,10.783
2,[Fantom] USDC-DAI,2021-03-05,3772.514951,48,2172.597593,6.517793,63.061
1,[Fantom] USDC-DAI,2021-03-04,3696.016205,43,865.346072,2.596038,25.637


In [28]:
criteria1 = df4_fantom['reserveUSD'] > 1
criteria2 = df4_fantom['date'] == df4_fantom['date'].max()

latest_results = df4_fantom.loc[(criteria1), :].sort_values(by = 'date',
                                 ascending = False)\
                    .drop_duplicates(subset = ['name'])\
                    .sort_values(by = '1y APR for 100 invested',
                                 ascending = False)\
                    .reset_index(drop = True)[['date', 'name', '1y APR for 100 invested',  'reserveUSD', 'volumeUSD', 'fee']]

df_styled  = latest_results.style.format({'reserveUSD': "{:0<4,.2f}",
                             'volumeUSD': "{:0<4,.2f}",
                             'fee': "{:0<4,.2f}",
                             '1y APR for 100 invested': "{:0<4,.2f}"})\
              .set_properties(subset=["name"], **{'text-align': 'left'})

df_styled

,date,name,1y APR for 100 invested,reserveUSD,volumeUSD,fee
0,2021-06-18 00:00:00,[Fantom] USDC-DAI,1.99,274.62,4.99,0.01
